In [3]:
import process as pr
import model as md
import pandas as pd
import forest as fr
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%reload_ext autoreload

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# the raw training data
raw = pr.get_raw('train')
# the processed training data
train_p = pr.get_post('train')
# the random forest
rf = fr.get_forest(train_p)
# the processed test data
test_p = pr.get_post('test')
# predicting the probabilities and aggregating the data
final = md.prep_for_submission(rf, test_p)
# saving the file to csv for upload
final.to_csv('final.csv', index=False)

In [9]:
X = train.drop('TripType', axis=1)
y = train.TripType # equivalent y = train_p.loc[:, 'TripType']

In [248]:
# without some adjustment, k nearest neighbors won't work well -- the classes are highly skewed
perc = train_p.groupby('TripType').size() / train_p.shape[0] * 100.0
pd.DataFrame(perc, columns=['percDist']).sort_values(by='percDist'))

,percDist
TripType,
14,0.005409
23,0.050228
4,0.139246
19,0.183601
31,0.272775


In [11]:
from sklearn.cross_validation import train_test_split

In [5]:
train = pr.get_post('train')
rf = fr.get_forest(train)
test = pr.get_post('test')
final7 = md.prep_for_submission(rf=rf, test=test)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   12.3s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   48.0s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:  1.8min
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:  3.2min
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  4.0min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    2.0s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   12.6s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:   29.9s
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:   55.6s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  1.2min finished


In [145]:
train = pr.get_post('train')

In [12]:
train_x, test_x, labels_x, labels_y = train_test_split(X, y)

In [19]:
rf = fr.get_forest(train, n_estimators=100)
rf.oob_score_

[Parallel(n_jobs=2)]: Done   1 out of 100 | elapsed:    0.4s remaining:   41.8s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:   13.9s finished


0.64318843988032248

In [24]:
rf = fr.get_forest(train, n_estimators=1000)
rf.oob_score_

[Parallel(n_jobs=-1)]: Done   1 out of 1000 | elapsed:    0.6s remaining:  9.2min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  2.3min finished


0.64823162048106198

In [168]:
sizes = [100,500,1000]
scores = []
for i in sizes:
    rf = fr.get_forest(train, n_estimators=i)
    score = rf.oob_score_
    scores.append(score)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    3.2s finished
/Users/jm/miniconda/envs/newenv/lib/python2.7/site-packages/sklearn/ensemble/forest.py:379: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    6.3s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    9.8s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:   11.4s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   14.3s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   14.3s finished


In [170]:
rf = fr.get_forest(train, n_estimators=50)
score = rf.oob_score_
score    

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   18.9s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   18.9s finished


0.61974940879812901

In [186]:
scores = []
depths = [50, 100, 150]
for i in depths:
    rf = fr.get_forest(train, n_estimators=i)
    score = rf.oob_score_
    scores.append(score)
scores

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   10.9s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   10.9s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   10.3s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   20.8s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   10.5s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:   31.7s finished


[0.64173819882674188, 0.64355426645239677, 0.64496531180182648]

In [188]:
scores = []
depths = [50, 100, 150]
for i in depths:
    rf = fr.get_forest(train.drop(['ratioReturnsToPurchases'], axis=1), n_estimators=i)
    score = rf.oob_score_
    scores.append(score)
scores

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   11.4s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   11.4s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   10.7s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   21.2s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   10.4s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:   32.5s finished


[0.63894223859731636, 0.64389396255503728, 0.64480852898522323]

In [189]:
scores = []
depths = [10000]
for i in depths:
    rf = fr.get_forest(train, n_estimators=i)
    score = rf.oob_score_
    scores.append(score)
scores

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   11.1s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   42.2s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:  1.6min
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:  2.9min
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  3.6min finished


[0.64747383686747928]

In [191]:
scores = []
depths = [10000]
for i in depths:
    rf = fr.get_forest(train, n_estimators=i)
    score = rf.oob_score_
    scores.append(score)
scores

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   11.2s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   45.6s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:  1.7min
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:  3.0min
[Parallel(n_jobs=1)]: Done 1250 jobs       | elapsed:  4.6min
[Parallel(n_jobs=1)]: Done 1800 jobs       | elapsed:  6.5min
[Parallel(n_jobs=1)]: Done 2450 jobs       | elapsed:  8.7min
[Parallel(n_jobs=1)]: Done 3200 jobs       | elapsed: 11.4min
[Parallel(n_jobs=1)]: Done 4050 jobs       | elapsed: 14.5min
[Parallel(n_jobs=1)]: Done 5000 jobs       | elapsed: 18.2min
[Parallel(n_jobs=1)]: Done 6050 jobs       | elapsed: 22.1min
[Parallel(n_jobs=1)]: Done 7200 jobs       | elapsed: 26.4min
[Parallel(n_jobs=1)]: Done 8450 jobs       | elapsed: 31.1min
[Parallel(n_jobs=1)]: Done 9800 jobs       | elapsed: 36.1min
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed: 36.9min finished


[0.64661153137616112]

In [190]:
scores = []
depths = [1000]
for i in depths:
    rf = fr.get_forest(train, n_estimators=i)
    score = rf.oob_score_
    scores.append(score)
scores

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   14.9s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:  1.1min
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:  2.4min
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:  4.1min
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  5.1min finished


[0.64551405165993803]

In [173]:
scores

[0.52340636799540108,
 0.62692222265773001,
 0.63749199754373587,
 0.63547995139732683,
 0.63452618926299009]

In [169]:
scores

[0.56973569030167626,
 0.61017259175061078,
 0.62225793386378181,
 0.62922170396791177,
 0.63256640405544884]

In [155]:
def f(a,b):
    if a == -1 or b == -1:
        return False
    else:
        return True
print reduce(f, [-1, 2, 3], True)

True


In [156]:
test = pr.get_post('test')

In [25]:
final7 = md.prep_for_submission(rf=rf, test=test)
final7.shape

[Parallel(n_jobs=4)]: Done   1 out of 342 | elapsed:    0.1s remaining:   22.6s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:   36.9s finished


(95674, 39)

In [26]:
final7.to_csv('submis6.csv', index=False)

In [6]:
from sklearn.cross_validation import cross_val_score

In [157]:
X = train.drop(['TripType'], axis=1)
y = train.TripType

In [9]:
cross_val_score(rf, X, y)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   13.1s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   52.6s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:  2.0min
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:  3.9min
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  4.9min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   10.6s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   25.2s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:   45.2s
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:  1.4min
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  1.8min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   12.9s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   49.9s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:  1.9min
[P

array([ 0.5879901 ,  0.62746143,  0.58180392])

In [158]:
cross_val_score(rf, X, y)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   13.9s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   52.2s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:  2.0min
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:  3.7min
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  4.6min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    2.2s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:    9.3s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:   20.9s
[Parallel(n_jobs=1)]: Done 800 jobs       | elapsed:   36.8s
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   45.9s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   12.3s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:   48.9s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:  1.9min
[P

array([ 0.62039678,  0.64273172,  0.60997647])

In [7]:
imps = rf.feature_importances_

In [160]:
len(imps)

86

In [161]:
len(rf.col)

86

In [162]:
featImps = pd.DataFrame({"features":rf.col, "importance":imps})

In [163]:
featImps.sort_values(by='importance')

,features,importance
59,SEASONAL,0.000000
48,OTHER DEPARTMENTS,0.000155
61,SHEER HOSIERY,0.000156
43,MENSWEAR,0.000172
55,PLUS AND MATERNITY,0.000244
10,CAMERAS AND SUPPLIES,0.000248
36,LADIES SOCKS,0.000255
7,BOOKS AND MAGAZINES,0.000393
14,CONCEPT STORES,0.000427
1,ACCESSORIES,0.000479


In [22]:
from sklearn.grid_search import GridSearchCV
param_grid = { 
    'min_samples_split':[1, 2, 3, 4, 5, 6],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6]
}

CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 2)
CV_rfc.fit(train_x, labels_x)
print CV_rfc.best_params_

[Parallel(n_jobs=2)]: Done   1 out of 100 | elapsed:    0.1s remaining:   12.9s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    5.5s finished
[Parallel(n_jobs=2)]: Done   1 out of 100 | elapsed:    0.0s remaining:    3.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.2s finished
[Parallel(n_jobs=2)]: Done   1 out of 100 | elapsed:    0.1s remaining:   11.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    5.6s finished
[Parallel(n_jobs=2)]: Done   1 out of 100 | elapsed:    0.0s remaining:    3.5s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.2s finished
[Parallel(n_jobs=2)]: Done   1 out of 100 | elapsed:    0.1s remaining:   11.7s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    5.5s finished
[Parallel(n_jobs=2)]: Done   1 out of 100 | elapsed:    0.0s remaining:    2.6s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=2)]: Done   1 out of 100 | elapsed:    0.1s remaining:   10.7s
[Parallel(n_jobs=2)]

{'min_samples_split': 2, 'min_samples_leaf': 3}


In [14]:
CV_rfc.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'